In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui
import dbManagement as dbm

In [3]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [86]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [90]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [99]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL020/2023-05-04/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
frame2time had 2 more than suite2p output. This happens sometimes. I don't like it. I think it's because scanimage sends a TTL before starting the frame
Computing red cell features for ATL020/2023-05-04/702... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
C:\Users\andrew\Documents\localData\ATL020\2023-05-04\702


In [100]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [ ]:
def getRoiStackPosition(self, mode='weightedmean'):
        planeIdx = self.getPlaneIdx()
        stat = self.loadS2P('stat')
        lam = [s['lam'] for s in stat]
        ypix = [s['ypix'] for s in stat]
        xpix = [s['xpix'] for s in stat]
        if mode=='weightedmean':
            yc = np.array([np.sum(l*y)/np.sum(l) for l,y in zip(lam,ypix)])
            xc = np.array([np.sum(l*x)/np.sum(l) for l,x in zip(lam,xpix)])
        elif mode=='median':
            yc = np.array([np.median(y) for y in ypix])
            xc = np.array([np.median(x) for x in xpix])
        stackPosition = np.stack((xc,yc,planeIdx))
        return stackPosition

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [107]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL027','2023-08-10','701')

\\zaru.cortexlab.net\Subjects\ATL027\2023-08-10\701
C:\Users\andrew\Documents\localData\ATL027\2023-08-10\701


In [35]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - ??? 
# ---------------------
# - Add new entry GUI!!!

In [ ]:
vrdb = dbm.vrDatabase()

In [5]:
vrdb.tableData()[0]

['uSessionID',
 'mouseName',
 'sessionDate',
 'sessionID',
 'experimentType',
 'experimentID',
 'variableGain',
 'behavior',
 'imaging',
 'faceCamera',
 'vrEnvironments',
 'headPlateRotation',
 'numPlanes',
 'planeSeparation',
 'pockelsPercentage',
 'objectiveRotation',
 'vrRegistration',
 'suite2p',
 'suite2pQC',
 'redCellQC',
 'sessionQC',
 'scratchJustification',
 'logtime',
 'sessionNotes',
 'suite2pDate',
 'vrRegistrationDate']

In [109]:
df = vrdb.showTable()

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [38]:
vrdb.checkS2P()

In [21]:
vrdb.printRequiresS2P()

In [ ]:
xx = vrdb.getTable(ignoreScratched=False, imaging=True, **{"sessionQC":False})
vrdb.showTable(xx)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [77]:
for idx, row in vrdb.getTable().iterrows():
    vrs = vrdb.vrSession(row)
    s2pPath = vrs.suite2pPath()
    print(s2pPath)
    break

C:\Users\andrew\Documents\localData\ATL012\2023-01-18\701\suite2p


In [85]:
import datetime
datetime.datetime.fromtimestamp(s2pPath.stat().st_ctime)

datetime.datetime(2023, 1, 18, 13, 52, 37, 790563)

In [16]:
df = vrdb.getTable() # full dataframe
cUIDs = df['uSessionID'].tolist()
newBool = [False]*len(cUIDs)

with vrdb.openCursor(commitChanges=True) as cursor:
    cursor.executemany(vrdb.createUpdateManyStatement('redCellQC'), zip(newBool,cUIDs))

In [131]:
df = vrdb.getTable()
df['redCellQC']

0     True
1     True
2     True
3     True
4     True
      ... 
87    True
88    True
89    True
90    True
91    True
Name: redCellQC, Length: 89, dtype: bool

In [133]:
vrdb.createUpdateStatement('sessionQC',False,cUID)

'UPDATE sessiondb set sessionQC = False WHERE uSessionID = 201'

In [139]:
help(vrdb.createUpdateManyStatement)

Help on method createUpdateManyStatement in module dbManagement:

createUpdateManyStatement(field) method of dbManagement.vrDatabase instance
    to update a single field many times where the value for each uSessionID is provided as a list to cursor.executemany()
    usage: 
    >>> cursor.executemany(self.createUpdateManyStatement(<field>, [(val,uid),(val,uid),...]))



In [146]:
vrdb.showTable(vrdb.getTable(ignoreScratched=False))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [147]:
df = vrdb.getTable(ignoreScratched=False)
len(df)

92

In [164]:
nn = pd.isnull(df['scratchJustification'])
nn[nn].index

Index([ 0,  1,  2,  3,  4,  5,  7,  8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91],
      dtype='int64')

In [15]:
vrdb.checkSessionScratch(withDatabaseUpdate=False)

In [34]:
sd = vrdb.add_suite2pDate()

In [30]:
for s in sd: 
    print(s.strftime('%m/%d/%y %H:%M:%S'))
    break

01/18/23 13:52:37
